# First Jupyter Notebook: Tutorial With DataSpell

This is my first attempt at using Jupyter notebook.

My goal is to learn fast.ai, but one must walk before running. Best to begin with a Jupyter notebook. I found a tutorial that introduces both markdown and calculations.

I also downloaded DataSpell, an early access IDE for data science from JetBrains. I think their IntelliJ IDE for Java is the best tool on the market. I wanted to see if their new tool could provide a similar boost for my venture into data science.

Every language has a “Hello World”. Jupyter notebook is no different. Let’s create a cell:

In [65]:
print('Hello World!')

Hello World!


In [66]:
import time
time.sleep(3)

So far, so good.  Adding cells with markdown and code.  This is a nice environment.  DataSpell is making it fun.  Let's add a function:

In [67]:
def say_hello(recipient):
    return 'Hello, {}!'.format(recipient)

say_hello('Michael')

'Hello, Michael!'

Jupyter notebook does a great job of keep code and documentation together.  It's a scientific revolution.  I think this is the best hope we have for maintaining that scientific ideal of making results public and easily reproducible by others.

I'll have to dig into [LaTeX](https://www.mathjax.org/) for math typesetting.  What does that look like?

Euler's identity: $$e^{i\pi} + 1 = 0$$<br>
$$\frac{arg 1}{arg 2}$$<br>
$$x^2$$<br>
$$e^{i\pi}$$<br>
$$A_i$$<br>
$$B_{ij}$$<br>
$$\sqrt[n]{arg}$$<br>

Here's a linear equation:

$$ \hat{Y} = \hat{\beta_0} + \sum_{j=1}^p \hat{\beta}_j X_j $$<br>

The tutorial has a bunch of markdown examples that aren't hard to understand.  No sense memorizing these.  I'll Google them when I need them.

# Level 1 heading
## Level 2 heading

Text is easy to format:

**bold**
_italic_

Lists are simple:

1. First element
2. Second element

* First bullet
* Second bullet

I created hyperlinks earlier.

It's easy to add code.  I hope to be able to add Python, R, and Kotlin someday soon.

`fun foo()`

Here's a code block:

```
fun bar() = "This could be some Kotlin code"
```

You can also tab in:

    fun baz():
        return 'A Python string'

What about images?  Let's add an image of a dog:

![Who dat?](../my_images/luna-napping.jpeg)

That's enough markdown.  This is about code.  Does DataSpell give me access to numpy?

In [68]:
import numpy as np

def square(q):
    return q*q

x = np.random.randint(1, 10)
y = square(x)
print('%d squared is %d' % (x, y))

8 squared is 64


Now I have numpy.   The tutorial tells me that there are kernels for over 100 languages, including Java, C, Fortran, R, and Kotlin.  There's an SoS kernel that provides multi-language support within a single notebook.  I'll stick to Python for now, but I can't wait to try out both R and Kotlin.

Enough about notebook mechanics.  What about a real analysis?

The tutorial problem uses a Fortune 500 data set.


In [69]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

!pwd

df = pd.read_csv('../datasets/fortune500.csv')

df.head()

/home/michaelduffy/Projects/fast.ai/fatigue-failure/_notebooks


,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1
